In [21]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

In [4]:
clmns = [
    "Open 1-2", "Open 2-3", "Open 3-4", "Open 4-5", "Open 5-6", "Open 6-7", "Open 7-8", "Open 8-9", "Open 9-10", "Open 10-11", "Open 11-12", "Open 12-13", "Open 13-14", "Open 14-15", "Open 15-16",
    "Close 1-2", "Close 2-3", "Close 3-4", "Close 4-5", "Close 5-6", "Close 6-7", "Close 7-8", "Close 8-9", "Close 9-10", "Close 10-11", "Close 11-12", "Close 12-13", "Close 13-14", "Close 14-15", "Close 15-16",
    "High 1-2", "High 2-3", "High 3-4", "High 4-5", "High 5-6", "High 6-7", "High 7-8", "High 8-9", "High 9-10", "High 10-11", "High 11-12", "High 12-13", "High 13-14", "High 14-15", "High 15-16",
    "Low 1-2", "Low 2-3", "Low 3-4", "Low 4-5", "Low 5-6", "Low 6-7", "Low 7-8", "Low 8-9", "Low 9-10", "Low 10-11", "Low 11-12", "Low 12-13", "Low 13-14", "Low 14-15", "Low 15-16",
    "Volume 1-2", "Volume 2-3", "Volume 3-4", "Volume 4-5", "Volume 5-6", "Volume 6-7", "Volume 7-8", "Volume 8-9", "Volume 9-10", "Volume 10-11", "Volume 11-12", "Volume 12-13", "Volume 13-14", "Volume 14-15", "Volume 15-16",
    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def read_txt_list():
  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[: , 1:]        
    data = np.array(data)
    llst = [0, 1, 2, 3, 5]
    for i in range(16, data.shape[0]):
        grow = []
        srow = []

        for j in llst:
           srow.append([
                data[i-1][j] - data[i-2][j],
                data[i-2][j] - data[i-3][j],
                data[i-3][j] - data[i-4][j],
                data[i-4][j] - data[i-5][j],
                data[i-5][j] - data[i-6][j],
                data[i-6][j] - data[i-7][j],
                data[i-7][j] - data[i-8][j],
                data[i-8][j] - data[i-9][j],
                data[i-9][j] - data[i-10][j],
                data[i-10][j] - data[i-11][j],
                data[i-11][j] - data[i-12][j],
                data[i-12][j] - data[i-13][j],
                data[i-13][j] - data[i-14][j],
                data[i-14][j] - data[i-15][j],
                data[i-15][j] - data[i-16][j],
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [5]:
def download_data(symbols,periodd,intervall):
  
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100

    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 12:
             data.to_csv(f"/content/data/{symbol}.csv")
             
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     data = np.array(unattached_dfs[0])
     for i in unattached_dfs[1:]:
           data = np.append(data, np.array(i), axis=0)
        
     unattached_dfs = []
  
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [7]:
symbols = get_crypto_syms()
#symbols = read_txt_list()
#symbols = read_syms_from_txt()
symbols = ["btc-usd","eth-usd","trx-usd","xrp-usd","ltc-usd"]

print(f"Symbols : {len(symbols)}")
download_data(symbols,"1d","5m")
folder_name = extract_data(1)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Symbols : 5
Data Folder Removed
 
Files In Data : 5
Processing File:
1 2 3 4 5  
(270, 77)
(216, 75) (216, 2)
(54, 75) (54, 2)


In [8]:
model = Sequential()



model.add(Dense(512, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))


opt = tf.keras.optimizers.Adamax()

model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               38912     
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dense_5 (Dense)             (None, 512)               262656    
                                                                 
 dense_6 (Dense)             (None, 512)               2

In [9]:
model.fit(xTrain,yTrain,epochs=30,batch_size=8,validation_data=(xTest,yTest),shuffle=False)

Epoch 1/30
27/27 [==============================] - 3s 55ms/step - loss: 0.6684 - accuracy: 0.6204 - val_loss: 0.6521 - val_accuracy: 0.6111
Epoch 2/30
27/27 [==============================] - 1s 20ms/step - loss: 0.5802 - accuracy: 0.6620 - val_loss: 0.6208 - val_accuracy: 0.6111
Epoch 3/30
27/27 [==============================] - 1s 19ms/step - loss: 0.3673 - accuracy: 0.8565 - val_loss: 0.7237 - val_accuracy: 0.6481
Epoch 4/30
27/27 [==============================] - 1s 19ms/step - loss: 0.1398 - accuracy: 0.9444 - val_loss: 0.7044 - val_accuracy: 0.7407
Epoch 5/30
27/27 [==============================] - 1s 19ms/step - loss: 0.0837 - accuracy: 0.9722 - val_loss: 1.3412 - val_accuracy: 0.6852
Epoch 6/30
27/27 [==============================] - 1s 20ms/step - loss: 0.1138 - accuracy: 0.9630 - val_loss: 1.9134 - val_accuracy: 0.6111
Epoch 7/30
27/27 [==============================] - 1s 20ms/step - loss: 0.3168 - accuracy: 0.8935 - val_loss: 0.6429 - val_accuracy: 0.6667
Epoch 8/30
27

In [13]:
data = yf.download("trx-usd",period="1d",interval="5m")
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-04-27 00:00:00+00:00,0.061926,0.061926,0.061871,0.061871,0.061871,0
2022-04-27 00:05:00+00:00,0.061900,0.061982,0.061900,0.061977,0.061977,1606464
2022-04-27 00:10:00+00:00,0.061956,0.062089,0.061956,0.062075,0.062075,611264
2022-04-27 00:15:00+00:00,0.062080,0.062080,0.062053,0.062062,0.062062,1377920
2022-04-27 00:20:00+00:00,0.062061,0.062084,0.062061,0.062084,0.062084,103488
...,...,...,...,...,...,...
2022-04-27 05:25:00+00:00,0.062855,0.062869,0.062850,0.062869,0.062869,439168
2022-04-27 05:30:00+00:00,0.062861,0.062890,0.062861,0.062879,0.062879,213760
2022-04-27 05:35:00+00:00,0.062890,0.062909,0.062890,0.062908,0.062908,256576


In [16]:
def process_for_prediction(data):
    i = -1
    row = []
    if len(pd.DataFrame(data).columns) == 7:
      data = data.iloc[: , 1:]        
    data = np.array(data)
    llst = [0, 1, 2, 3, 5]
    grow = []
    srow = []
    for j in llst:
           srow.append([
                data[i-1][j] - data[i-2][j],
                data[i-2][j] - data[i-3][j],
                data[i-3][j] - data[i-4][j],
                data[i-4][j] - data[i-5][j],
                data[i-5][j] - data[i-6][j],
                data[i-6][j] - data[i-7][j],
                data[i-7][j] - data[i-8][j],
                data[i-8][j] - data[i-9][j],
                data[i-9][j] - data[i-10][j],
                data[i-10][j] - data[i-11][j],
                data[i-11][j] - data[i-12][j],
                data[i-12][j] - data[i-13][j],
                data[i-13][j] - data[i-14][j],
                data[i-14][j] - data[i-15][j],
                data[i-15][j] - data[i-16][j],
           ])

    for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        
    arr = np.array(grow).flatten()
    row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)
def make_prediction(symbol,timeframe):
  if timeframe == "5m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))    
  if timeframe == "15m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1h":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1d":
    data = yf.download(symbol,period="20d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1wk":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "2m":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "30m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "5d":
    data = yf.download(symbol,period="5mo",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "90m":
    data = yf.download(symbol,period="5d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "3mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  

In [17]:
make_prediction("trx-usd","5m")

[*********************100%***********************]  1 of 1 completed


array([[8.225143e-04, 9.998231e-01]], dtype=float32)

In [28]:
tv = TvDatafeed()
tv.get_hist(symbol='TRXUSDT',exchange='BINANCE',interval=Interval.in_5_minute,n_bars=30)

,symbol,open,high,low,close,volume
datetime,,,,,,
2022-04-27 03:25:00,BINANCE:TRXUSDT,0.06263,0.06264,0.06257,0.06258,2061266.9
2022-04-27 03:30:00,BINANCE:TRXUSDT,0.06259,0.06261,0.06247,0.06260,1849872.2
2022-04-27 03:35:00,BINANCE:TRXUSDT,0.06260,0.06261,0.06254,0.06257,1510140.0
2022-04-27 03:40:00,BINANCE:TRXUSDT,0.06256,0.06257,0.06244,0.06247,1886170.3
2022-04-27 03:45:00,BINANCE:TRXUSDT,0.06246,0.06250,0.06232,0.06237,4534167.0
2022-04-27 03:50:00,BINANCE:TRXUSDT,0.06238,0.06245,0.06236,0.06239,2950322.6
2022-04-27 03:55:00,BINANCE:TRXUSDT,0.06239,0.06244,0.06235,0.06243,1285161.3
2022-04-27 04:00:00,BINANCE:TRXUSDT,0.06244,0.06258,0.06242,0.06258,2453782.8
2022-04-27 04:05:00,BINANCE:TRXUSDT,0.06258,0.06270,0.06257,0.06267,3556572.9


In [23]:
def process_for_prediction(data):
    i = -1
    row = []
    data.drop("symbol",axis=1,inplace=True)   
    data = np.array(data)
    llst = [0, 1, 2, 3, 4]
    grow = []
    srow = []
    for j in llst:
           srow.append([
                data[i-1][j] - data[i-2][j],
                data[i-2][j] - data[i-3][j],
                data[i-3][j] - data[i-4][j],
                data[i-4][j] - data[i-5][j],
                data[i-5][j] - data[i-6][j],
                data[i-6][j] - data[i-7][j],
                data[i-7][j] - data[i-8][j],
                data[i-8][j] - data[i-9][j],
                data[i-9][j] - data[i-10][j],
                data[i-10][j] - data[i-11][j],
                data[i-11][j] - data[i-12][j],
                data[i-12][j] - data[i-13][j],
                data[i-13][j] - data[i-14][j],
                data[i-14][j] - data[i-15][j],
                data[i-15][j] - data[i-16][j],
           ])

    for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        
    arr = np.array(grow).flatten()
    row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)
def make_prediction(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='TRXUSDT',exchange='BINANCE',interval=Interval.in_5_minute,n_bars=30)
   return model.predict(process_for_prediction(data))

In [29]:
make_prediction("s","x")

array([[9.9994200e-01, 2.1633506e-04]], dtype=float32)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
